In [1]:
# These are my default settings
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import joblib

plt.rcParams["figure.figsize"] = (12, 6)
sns.set()
pd.set_option("display.max_columns", None)

import zipfile
import shutil

In [2]:
shutil.unpack_archive('Data.zip', 'Mednist')

# load test data, trained CNN model

In [1]:
test_dir = input("Enter your file path")

Enter your file path 


In [ ]:
from tensorflow.keras import models

model = models.load_model("cnn_model")
model.summary()

# Preprocessing

In [ ]:
# Get batch size

import os
import glob

def get_full_batch_size(data_dir):
    image_files = []
    for class_dir in os.listdir(data_dir):
        class_image_files = glob.glob(os.path.join(data_dir, class_dir, '*'))
        image_files.extend(class_image_files)
    return len(image_files)


In [ ]:
get_full_batch_size(test_dir)

In [ ]:
# setting get image

BATCH_SIZE = get_full_batch_size(test_dir)
IMAGE_SIZE = (64, 64)
RANDOM_STATE = 42

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Rescaling

normalization_layer = tf.keras.layers.Rescaling(1./255)

def preprocessor(data_dir):
    data = image_dataset_from_directory(
        data_dir,
        seed=RANDOM_STATE,
        image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE
    )
    c_names = data.class_names
    normalized_ds = data.map(lambda x, y: (normalization_layer(x), y))
    images, labels = next(iter(normalized_ds))
    
    return normalized_ds, images, labels, c_names


In [ ]:
test_data, X, y, CLASS_NAMES = preprocessor(test_dir)

In [ ]:
print(X.shape, y.shape)

In [ ]:
CLASS_NAMES

# Prediction and evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def prediction(model, features, labels):
    pred = model.predict(features)
    pred_labels = np.argmax(pred, axis=1)
    return pred_labels

In [ ]:
y_pred = prediction(model, X, y)

In [ ]:
print(classification_report(y, y_pred, target_names=CLASS_NAMES))


cm = confusion_matrix(y, y_pred)

print(cm)
sns.heatmap(cm, annot=True, fmt='g', cmap='Spectral')

In [3]:
shutil.rmtree("Data")